In [1]:
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')


# Load environment variables from the .env file
load_dotenv()

True

In [15]:
def elastic_search(es_client, query, limit=3, filter=None, random=False):
    if random:
        search_query = {
          "size": 10,
          "query": {
            "function_score": {
              "functions": [
                {
                  "random_score": {}
                }
              ]
            }
          }
        }
    else:
        search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["relief", "positive_effects", "flavours"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {  # TODO: add filter as parameter
                        "term": {
                            "category": "flower"
                        }
                    }
                }
            }
        }
    response = es_client.search(index=index_name, body=search_query)
    
    return response['hits']['hits'][:limit]

def pretty(search_results):
    result_docs = []
    
    for hit in search_results:
        result_docs.append({hit['_score']: hit['_source']})
    return result_docs


query = 'Lemon, headache'
index_name = "course-questions"

res = elastic_search(es_client, query, random=True)
sample = res[0]['_source']
print(sample.keys())
print()
print(pretty(res))

dict_keys(['title', 'tags', 'relief', 'positive_effects', 'flavours', 'avg_rating', 'num_ratings'])

[{0.9979122: {'title': 'White Widow', 'tags': 'Sativa Dominant|60% Sativa/40% Indica', 'relief': 'bipolar disorder chronic pain depression fatigue headaches migraines muscle spasms ptsd', 'positive_effects': 'creative euphoria happy uplifting', 'flavours': 'dank pine spicy sweet woody', 'avg_rating': 4.4, 'num_ratings': 112}}, {0.9906636: {'title': 'Pink Kush', 'tags': 'Indica Dominant|90% Indica/10% Sativa', 'relief': 'anxiety bipolar disorder chronic pain depression inflammation insomnia loss of appetite migraines ptsd stress', 'positive_effects': 'body high euphoria happy hungry relaxing sleepy', 'flavours': 'berry candy sweet vanilla', 'avg_rating': 4.7, 'num_ratings': 50}}, {0.9890757: {'title': 'Lemon Kush', 'tags': 'Hybrid,|50%/50%', 'relief': 'bipolar disorder chronic pain depression headaches insomnia migraines ptsd stress', 'positive_effects': 'creative euphoria happy relaxing

In [17]:
import os

current_file_dir = os.path.dirname(__file__)
print(current_file_dir)

NameError: name '__file__' is not defined

In [20]:
current_file_dir = os.getcwd()
os.listdir(os.path.join(os.path.dirname(current_file_dir), 'data', 'pipelines-data'))

['content_reviews_green_bro.json',
 'es-data',
 'content_green_bro.csv',
 'dagster-data']